In [ ]:
from mumaxplus import *
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
@np.vectorize
def expectation_mz_langevin(msat, bext, temperature, cellvolume):
    kB = 1.381e-23
    xi = msat*cellvolume*bext/(kB*temperature)
    return 1/np.tanh(xi) - 1/xi

In [ ]:
msat = 800e3
cellvolume = 100e-27
bext = 0.05
temperatures = np.linspace(1, 500, 25)

N = 1024
relaxtime = 1e-9
sampletime = 1e-9
nsamples = 200

world = World(cellsize=3*[np.power(cellvolume, 1./3.)])
magnet = Ferromagnet(world, Grid((N, 1, 1)))
magnet.enable_demag = False
magnet.aex = 0.0
magnet.alpha = 0.1
magnet.msat = msat
magnet.magnetization = (0,0,1) # groundstate

solver = world.timesolver

@np.vectorize
def expectation_mz_simul(bext, temperature):
    world.bias_magnetic_field = (0, 0, bext)
    magnet.temperature = temperature
    solver.run(relaxtime)
    outputquantities = {"mz": lambda: magnet.magnetization.average()[2]}
    timepoints = solver.time + np.linspace(0, sampletime, nsamples)
    output = world.timesolver.solve(timepoints, outputquantities)
    return np.average(output['mz'])

m_simul = expectation_mz_simul(bext, temperatures)
m_langevin = expectation_mz_langevin(msat, bext, temperatures, cellvolume)

plt.plot(temperatures, m_simul, 'o', label="Simulation")
plt.plot(temperatures, m_langevin, 'k-', label="theory")
plt.xlabel("Temperature (K)")
plt.ylabel("<$m_z$>")
plt.legend()
plt.show()

In [ ]:
temperature = 200
bexts = np.linspace(0.2,0.05,20)

magnet.magnetization = (0,0,1) # groundstate

m_simul = expectation_mz_simul(bexts, temperature)
m_langevin = expectation_mz_langevin(msat, bexts, temperature, cellvolume)

plt.plot(bexts, m_simul, 'o', label="Simulation")
plt.plot(bexts, m_langevin, 'k-', label="theory")
plt.xlabel(r"$B_{\rm ext}$ (T)")
plt.ylabel("<$m_z$>")
plt.legend()
plt.show()